### Installation

You will need to install both the `langchain_community` pacakges:

In [2]:
%pip install -qU asyncio langchain_community

Note: you may need to restart the kernel to use updated packages.


# Nimble AnswerIt Retriever


In [3]:
import getpass
import os

if "NIMBLE_API_KEY" not in os.environ:
    os.environ["NIMBLE_API_KEY"] = getpass.getpass("Enter your NIMBLE_API_KEY: ")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

### Instantiation
Now we can instantiate our retriever, initializing a client and optionally pre-adding data sources urls

In [4]:
from langchain_community.retrievers.nimbleit import (
    NimbleItRetriever,
)

retriever = NimbleItRetriever(
    answerit_url="https://answerit.webit.live",
    authorization_token=os.environ["NIMBLE_API_KEY"],
)

In [5]:
print('pipeline_id', retriever.store.pipeline_id)

pipeline_id 4be8b76e-d62a-4b61-bd6c-1e7b1f9e5618


#### Add Sources of Information from URLs

In [6]:
retriever.add_sources(
    [
        {
            "url": "https://lilianweng.github.io/posts/2023-06-23-agent/",
            "return_most_relevant_content": False,
            "depth": 0
        },
        {
            "url": "https://en.wikipedia.org/wiki/Pizza",
            "return_most_relevant_content": False,
            "depth": 0
        }
    ],
    wait=True
 )

In [7]:
result = retriever.invoke("What is llm?")
print(result)

[Document(metadata={'evidences': [{'evidence': 'LLM-powered Autonomous Agents', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'Current agent system relies on natural language as an interface between LLMs and external components such as memory and tools.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'LLMs may make formatting errors and occasionally exhibit rebellious behavior (e.g. refuse to follow an instruction).', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'Both TALM (Tool Augmented Language Models; [Parisi et al. 2022](https://arxiv.org/abs/2205.12255)) and

### Use within a chain
Initialize ChatOpenAI and define a prompt template



In [8]:
!pip install langchain_openai

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


llm = ChatOpenAI()

PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.

<context>
{context}
</context>

<question>
{question}
</question>

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

#### Define a function for formatting documents



In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### Define a chain using the retriever and other components

In [11]:
retriever = NimbleItRetriever(
    answerit_url="https://answerit.webit.live",
    authorization_token=os.environ["NIMBLE_API_KEY"],
)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### Add sources of information

In [12]:
retriever.add_sources([
    {'url': 'https://en.wikipedia.org/wiki/Lila_(Robinson_novel)', 'depth': 0},
    {'url': 'https://www.newyorker.com/magazine/2014/10/06/lonesome-road', 'depth': 0}
], wait=True)

#### Perform a query using the defined chain

In [13]:
result = rag_chain.invoke("What novels has Lila written and what are their contents?")
print(result)

Lila is not an author; rather, she is a character in Marilynne Robinson's novels. Marilynne Robinson has written several novels, including 'Lila,' which is the third installment in her Gilead series. The novels in the Gilead series are 'Gilead' (2004), 'Home' (2008), and 'Lila' (2014). 'Lila' focuses on the courtship and marriage of Lila and John Ames, as well as Lila's transient past and her complex attachments. The novel explores themes such as suffering, abandonment, forgiveness, and rescue. It also delves into Lila's feelings of shame over her poverty, lack of beauty, and ignorance, and her struggle with loneliness even after her marriage to Ames.
